In [1]:
# Read Data
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
ticker_names = ["MSFT", "AAPL", "NVDA", "AMZN", "GOOG", \
           "GOOGL", "META", "TSM", "LLY", "AVGO", \
            "NVO", "TSLA", "JPM", "V", "WMT", \
                "UNH", "MA", "XOM", "ASML", "JNJ"]
dfs = []
X = []
Y = []



time_steps = 7

for name in ticker_names:  
    df = pd.read_csv(f'../DATA/Cleaned Datas/cleaned_{name}_prices.csv')
    df = df.drop(columns=['Date'])
    df = df.drop(columns=['Volume'])
    scaler_of_df = MinMaxScaler(feature_range=(0,1))
    df_scaled = scaler_of_df.fit_transform(df)
    dfs.append(df_scaled)

    
    open = np.array(df_scaled[:, 0])
    high = np.array(df_scaled[:, 1])
    low = np.array(df_scaled[:, 2])
    close = np.array(df_scaled[:, 3])

    X_name = []
    Y_name = []
    

    for i in range(len(open)-time_steps-1):
        X_i = np.vstack((open[i:i+time_steps], high[i:i+time_steps], low[i:i+time_steps])).T
        Y_i = close[i+time_steps]
        X_name.append(X_i)
        Y_name.append(Y_i)

    X_name = np.array(X_name)
    Y_name = np.array(Y_name)

    X.append(X_name)
    Y.append(Y_name)

X = np.array(X) # 3d arr: num_ticker * num_sample * num_features
Y = np.array(Y) # 2d arr: num_ticker * num_sample

X_trains = [] # 3d arr: ticker * sample * features
Y_trains = [] # 2d arr: ticker * sample
X_cvs = [] # 3d arr: ticker * sample * features
Y_cvs = [] # 2d arr: ticker * sample
X_tests = [] # 3d arr: ticker * sample * features
Y_tests = [] # 2d arr: ticker * sample

training_sizes = []
cv_sizes = []
test_sizes = []

for ticker in X:
    train_size = int(len(ticker) * 0.6)
    cv_size = int(len(ticker) * 0.2)
    test_size = len(ticker) - train_size - cv_size

    training_sizes.append(train_size)
    cv_sizes.append(cv_size)
    test_sizes.append(test_size)

    X_train = ticker[:train_size]
    X_cv = ticker[train_size:train_size+cv_size]
    X_test = ticker[train_size+cv_size:]

    X_trains.append(X_train)
    X_cvs.append(X_cv)
    X_tests.append(X_test)


for ticker in Y:
    train_size = int(len(ticker) * 0.6)
    cv_size = int(len(ticker) * 0.2)

    Y_train = ticker[:train_size]
    Y_cv = ticker[train_size:train_size+cv_size]
    Y_test = ticker[train_size+cv_size:]

    Y_trains.append(Y_train)
    Y_cvs.append(Y_cv)
    Y_tests.append(Y_test)


X_trains = np.array(X_trains) # 3d arr: ticker * sample * features
Y_trains = np.array(Y_trains) # 2d arr: ticker * sample
X_cvs = np.array(X_cvs) # 3d arr: ticker * sample * features
Y_cvs = np.array(Y_cvs) # 2d arr: ticker * sample
X_tests = np.array(X_tests) # 3d arr: ticker * sample * features
Y_tests = np.array(Y_tests) # 2d arr: ticker * sample


/var/folders/c1/t6c91th93rg_t8cmmzfjwp840000gn/T/ipykernel_3962/1596732867.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X) # 3d arr: num_ticker * num_sample * num_features
/var/folders/c1/t6c91th93rg_t8cmmzfjwp840000gn/T/ipykernel_3962/1596732867.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.array(Y) # 2d arr: num_ticker * num_sample
/var/folders/c1/t6c91th93rg_t8cmmzfjwp840000gn/T/ipykernel_3962/1596732867.py:92: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple o

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Activation, Dropout
from tensorflow.keras.losses import mse, mae
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.preprocessing import MinMaxScaler




time_steps = 7 # a week of steps
input_dim = 3 # OHL

# # Initialize an empty list to store the reshaped sequences
# X_train_reshaped = []

# # Calculate the new number of samples based on the time steps
# num_sequences = len(X_train) - time_steps + 1

# # Reshape the data
# for i in range(num_sequences):
#     # Extract a sequence of length `time_steps` starting from index i
#     sequence = X_train[i:i + time_steps, :]
#     # Append the sequence to your new list
#     X_train_reshaped.append(sequence)



for i in range(len(ticker_names)):
    x_t = X_trains[i]
    y_t = Y_trains[i]
    x_cv = X_cvs[i]
    y_cv = Y_cvs[i]



    model = Sequential([   
        LSTM(units= 256, activation='relu', return_sequences=True, input_shape=(time_steps, input_dim)),
        Dropout(0.1),
        LSTM(units= 128, activation='relu', return_sequences=False),
        Dropout(0.1),
        Dense(units= 64, activation='relu'),
        Dropout(0.1),
        Dense(units= 1, activation='linear'),
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    model.fit(x_t, y_t, validation_data=(x_cv, y_cv) , epochs=100, batch_size=64)
    model.save(f'{ticker_names[i]}.h5')


Epoch 1/100


2024-03-13 11:00:07.800711: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


47/90 [==============>...............] - ETA: 0s - loss: 5.4845e-04

KeyboardInterrupt: 